In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import missingno as msno

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

In [ ]:
!pip install pyodbc pandas sqlalchemy


**1) Ver los parametros del dataset**

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/Analitica Trabajo/Colombianos.csv')

df.tail()


,País,Código ISO país,Oficina de registro,Grupo edad,Edad (años),Área Conocimiento,Sub Area Conocimiento,Nivel Académico,Estado civil,Género,Etnia de la persona,Estatura (CM),Localización,Cantidad de personas
788753,ALEMANIA,DEU,C. FRANKFURT,ADULTO,31,"ECONOMÍA, ADMINISTRACIÓN CONTADURIA Y AFINES",ADMINISTRACIÓN,PREGRADO - PROFESIONAL,SOLTERO,FEMENINO,OTRO,-1,"(51.165691, 10.451526)",2
788754,ESTADOS UNIDOS,USA,C. NEWARK,ADULTO MAYOR,62,"ECONOMÍA, ADMINISTRACIÓN CONTADURIA Y AFINES",ADMINISTRACIÓN,NO INDICA,CASADO,FEMENINO,OTRO,-1,"(37.09024, -95.712891)",14
788755,ESPAÑA,ESP,C. MADRID,ADULTO,46,NINGUNA,NINGUNA,PREGRADO - TÉCNICO PROFESIONAL,CASADO,FEMENINO,OTRO,163,"(40.463667, -3.74922)",1
788756,ESPAÑA,ESP,C. BARCELONA,ADULTO,41,MATEMÁTICAS Y CIENCIAS NATURALES,QUÍMICA Y AFINES,PREGRADO - TECNOLÓGICO,SOLTERO,FEMENINO,OTRO,-1,"(40.463667, -3.74922)",1
788757,CHILE,CHL,C. ANTOFAGASTA,ADULTO,51,NINGUNA,NINGUNA,BACHILLERATO,CASADO,FEMENINO,OTRO,-1,"(-35.675147, -71.542969)",2


**1. Normalización de la columna Localización**

In [ ]:
df[['Latitud', 'Longitud']] = df['Localización'].str.extract(r'\(([^,]+),\s([^)]+)\)')
df['Latitud'] = pd.to_numeric(df['Latitud'], errors='coerce')
df['Longitud'] = pd.to_numeric(df['Longitud'], errors='coerce')

In [ ]:
# Eliminar la columna original 'Localización'
df.drop('Localización', axis=1, inplace=True)

**2. Eliminar registros con valores nulos**

In [ ]:
# Verificar si hay registros nulos y eliminarlos
df.replace(-1, np.nan, inplace=True)  # Considerar -1 como NaN para estaturas
df.dropna(inplace=True)  # Eliminar filas con valores nulos


**3. Mostrar el DataFrame final después de la normalización y limpieza**

In [ ]:
print("\nDataFrame normalizado y sin valores nulos:")
print(df)


DataFrame normalizado y sin valores nulos:
               País Código ISO país   Oficina de registro    Grupo edad  \
1            ESPAÑA             ESP          C. BARCELONA        ADULTO   
4       REINO UNIDO             GBR            C. LONDRES        ADULTO   
9         VENEZUELA             VEN       C. PUERTO ORDAZ  ADULTO JOVEN   
12        VENEZUELA             VEN            C. CARACAS  ADULTO MAYOR   
16           ESPAÑA             ESP             C. MADRID  ADULTO JOVEN   
...             ...             ...                   ...           ...   
788732  EL SALVADOR             SLV       C. SAN SALVADOR  ADULTO MAYOR   
788740       ESPAÑA             ESP  C. LAS PALMAS DE G.C        ADULTO   
788742       ESPAÑA             ESP             C. MADRID        ADULTO   
788749       ESPAÑA             ESP  C. LAS PALMAS DE G.C        ADULTO   
788755       ESPAÑA             ESP             C. MADRID        ADULTO   

        Edad (años)     Área Conocimiento Sub Area Cono

**Crear índices en la base de datos SQL para optimizar consultas.**

In [ ]:
server = 'DESKTOP-9JC24GI'  # Cambia esto por tu servidor
database = 'colombiano'          # Nombre de tu base de datos
username = 'DESKTOP-9JC24GI\\PERSONAL'          # Tu usuario de SQL Server
password = ''       # Tu contraseña de SQL Server

# Usando pyodbc + sqlalchemy para conectarse a SQL Server
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

# Paso 4: Crear la tabla en SQL Server (se ejecutará automáticamente si no existe)
# El método to_sql crea la tabla si no existe, y carga los datos
df.to_sql('personas', engine, if_exists='replace', index=False)

print("Datos cargados en la tabla 'personas' de la base de datos 'colombiano'")

DBAPIError: (pyodbc.Error) ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/dbapi)